# Simulation de la température d'un câble électrique

Ce notebook présente une simulation de l'évolution de la température d'un câble électrique en fonction de différents paramètres physiques (température ambiante, vitesse du vent, intensité du courant).
Il utilise des équations différentielles et la bibliothèque Plotly pour visualiser les résultats.

**Plan du notebook :**
1. Import des bibliothèques
2. Définition de l'équation différentielle
3. Fonction de simulation sur une période donnée
4. Simulation répétée sur plusieurs minutes
5. Visualisation des résultats
6. Exécution d'un exemple complet

## 1. Import des bibliothèques


In [ ]:
import time
from typing import List, Tuple

import numpy as np
import plotly.graph_objects as go
from scipy.integrate import odeint

## 2. Définition de l'équation différentielle

La fonction `d_tc_dt` modélise la variation de la température du câble (`tc`) en fonction du temps.
Elle prend en compte :
- la température ambiante (`ta`)
- la vitesse du vent (`ws`)
- l'intensité du courant (`i`)

L'équation utilisée est empirique et permet de simuler le refroidissement ou le réchauffement du câble.

In [ ]:
def d_tc_dt(tc: float, t: float, ta: float, ws: float, i: float) -> float:
    """
    Calcule la dérivée de la température du câble à un instant donné.
    :param tc: Température actuelle du câble (°C)
    :param t: Temps (s)
    :param ta: Température ambiante (°C)
    :param ws: Vitesse du vent (m/s)
    :param i: Intensité (A)
    :return: Dérivée de la température du câble (°C/s)
    """
    a: float = ((ws ** 2) / 1600) * 0.4 + 0.1  # Influence du vent
    b: float = ((i ** 1.4) / 73785) * 130      # Influence du courant
    return -(1 / 60) * a * (tc - ta - b)

## 3. Fonction de simulation sur une période donnée

La fonction `simulate_cable_temp` simule l'évolution de la température du câble sur une durée définie.
Elle utilise la méthode d'intégration numérique `odeint` pour résoudre l'équation différentielle.

**Paramètres principaux :**
- `ta` : température ambiante
- `ws` : vitesse du vent
- `i` : intensité du courant
- `tc_initial` : température initiale du câble
- `simulation_time_min` : durée de la simulation (en secondes)
- `microsecond_step` : pas de temps pour l'intégration (plus petit = plus précis)

In [ ]:
def simulate_cable_temp(
        ta: float,
        ws: float,
        i: float,
        tc_initial: float,
        simulation_time_min: int = 60,
        microsecond_step: float = 1e-6
) -> Tuple[float, float]:
    """
    Simule la température du câble sur une période donnée.
    :param ta: Température ambiante (°C)
    :param ws: Vitesse du vent (m/s)
    :param i: Intensité (A)
    :param tc_initial: Température initiale du câble (°C)
    :param simulation_time_min: Durée de la simulation (s)
    :param microsecond_step: Pas de temps pour la simulation (s)
    :return: Tuple contenant la température finale du câble et le temps d'exécution (s).
    """
    total_time_s: float = simulation_time_min
    t: np.ndarray = np.arange(0, total_time_s, microsecond_step)

    start_time: float = time.time()
    tc_sol: np.ndarray = odeint(d_tc_dt, tc_initial, t, args=(ta, ws, i), hmax=microsecond_step)
    end_time: float = time.time()

    final_tc: float = float(tc_sol[-1])

    return final_tc, end_time - start_time

## 4. Simulation répétée sur plusieurs minutes

La fonction `run_x_min_simulation` permet de simuler l'évolution de la température du câble minute par minute sur une durée totale (par exemple 30 minutes).
À chaque itération, la température finale devient la température initiale de la minute suivante.

Cela permet de visualiser l'évolution progressive de la température.


In [ ]:
def run_x_min_simulation(
        minutes: int = 30,
        step: int = 1,
        microsecond_step: float = 1e-6
) -> Tuple[List[float], List[float]]:
    """
    Simule la température du câble sur plusieurs minutes, en répétant la simulation chaque minute.
    :param minutes: Nombre de minutes à simuler (par défaut 30).
    :param step: Pas de temps pour la simulation (s)
    :param microsecond_step: Pas de temps pour la simulation (s)
    :return: Liste des températures et temps d'exécution pour chaque minute.
    """
    ta: float = 25  # Température ambiante (°C)
    ws: float = 1   # Vitesse du vent (m/s)
    i: float = 300  # Intensité (A)
    tc_initial: float = 25  # Température initiale du câble

    tc_list: List[float] = []
    exec_times: List[float] = []

    tc_current: float = tc_initial

    minutes_seconde = minutes * 60
    for tmp in range(0, minutes_seconde, step):
        minute = (tmp + step) / 60
        print(f"Simulation de la minute {minute}...")
        tc: float
        exec_time: float
        tc, exec_time = simulate_cable_temp(
            ta, ws, i, tc_current,
            simulation_time_min=step,
            microsecond_step=microsecond_step
        )
        tc_list.append(tc)
        exec_times.append(exec_time)
        tc_current = tc  # Utiliser la temp finale comme nouvelle initiale

        print(
            f"Minute {minute}: Tc = {tc:.2f}°C, "
            f"Temps = {exec_time:.2f} s"
        )

    return tc_list, exec_times

## 5. Version simplifiée de la simulation

La fonction `run_x_min_simulation_simple` permet de lancer la simulation en spécifiant le pas en minutes (au lieu de secondes).
Elle convertit simplement le pas en secondes et appelle la fonction précédente.

In [ ]:
def run_x_min_simulation_simple(
        minutes: int = 30,
        step: int = 1,
        microsecond_step: float = 1e-6
) -> Tuple[List[float], List[float]]:
    """
    Simule la température du câble sur plusieurs minutes, avec un pas en minutes.
    :param minutes: Nombre de minutes à simuler
    :param step: Pas de temps pour la simulation (minutes)
    :param microsecond_step: Pas de temps pour la simulation (s)
    :return: Liste des températures et temps d'exécution pour chaque minute.
    """
    return run_x_min_simulation(
        minutes=minutes,
        step=step * 60,  # Convertir les minutes en secondes
        microsecond_step=microsecond_step
    )

## 6. Visualisation des résultats

La fonction `plot_results` utilise Plotly pour afficher l'évolution de la température du câble au fil du temps.
Le graphique est interactif et permet d'explorer les valeurs obtenues.

In [ ]:
def plot_results(
        tc_list: List[float],
        minutes: int = 30
) -> None:
    """
    Affiche les résultats de la simulation sous forme de graphique interactif.
    :param tc_list: liste des températures du câble
    :param minutes: Nombre de minutes simulées
    """
    fig: go.Figure = go.Figure()
    fig.add_trace(go.Scatter(y=tc_list, mode='lines+markers', name='Température câble (°C)'))

    fig.update_layout(
        title=f"Simulation température câble sur {minutes} minutes",
        xaxis_title="Minute",
        yaxis_title="Température (°C)",
        template="plotly_dark"
    )

    fig.show()

## 7. Exécution d'un exemple complet

Nous allons maintenant exécuter une simulation sur 30 minutes, avec un pas de 1 minute, puis afficher le résultat.

**Paramètres de simulation :**
- Température ambiante (°C)
- Pas de temps pour la simulation (minutes)
- Pas de temps pour la simulation (s)
    - `0.01` pour une simulation plus rapide mais moins précise
    - `1e-6` pour une simulation plus précise mais plus lente

In [ ]:
minutes: int = 30

In [ ]:
step: int = 1

In [ ]:
microsecond_step: float = 0.01 # 1e-6

In [ ]:
tc_list, exec_times = run_x_min_simulation_simple(
    minutes=minutes, step=step,
    microsecond_step=microsecond_step
)
plot_results(tc_list, minutes)